# ADVANCED VISUALISATIONS WITH PLOTLY

In [55]:
import pandas as pd
from pathlib import os
import numpy as np
import plotly.graph_objects as go
import plotly.express as px

In [56]:
path = "../data"
files = {i.split(".")[0]:os.path.join(path, i) for i in os.listdir(path)}

In [57]:
files

{'procedures': '../data/procedures.csv',
 'allergies': '../data/allergies.csv',
 'medications': '../data/medications.csv',
 '': '../data/.ipynb_checkpoints',
 'immunizations': '../data/immunizations.csv',
 'providers': '../data/providers.csv',
 'payers': '../data/payers.csv',
 'devices': '../data/devices.csv',
 'careplans': '../data/careplans.csv',
 'payer_transitions': '../data/payer_transitions.csv',
 'imaging_studies': '../data/imaging_studies.csv',
 'observations': '../data/observations.csv',
 'patients': '../data/patients.csv',
 'encounters': '../data/encounters.csv',
 'organizations': '../data/organizations.csv',
 'conditions': '../data/conditions.csv'}

### BASIC DATA CLEANING

- Try Understanding the meaning of each file and creating a scenario out of it
- Its simple, a patient goes to a hospital and is attended by a healthcare provider. The patient is then treated for some given ailment. Certain medications and procedures are prescribed to the patient and most of the cost is covered by the patient's Insurance agency 

- Now try relating this to each of the csv files

- Patient.csv  ---> the patient details

- Organisation.csv ---> the hospital details

- Providers.csv ---> healthcare professional details

- payers.csv ---> Insurance agency details

- medications.csv ---> medicines and drug details

- procedures.csv ---> procedures / medical tests 

- devices.csv ---> devices used

- encounters.csv ---> the entire event itself

#### The encounter data is the crux for all the other data. Let's take a look at it

In [58]:
encounter=pd.read_csv(files["encounters"])
encounter.sample(3)

,Id,START,STOP,PATIENT,ORGANIZATION,PROVIDER,PAYER,ENCOUNTERCLASS,CODE,DESCRIPTION,BASE_ENCOUNTER_COST,TOTAL_CLAIM_COST,PAYER_COVERAGE,REASONCODE,REASONDESCRIPTION
43427,46637889-3827-4bdf-97a5-5036bf71cc4c,2019-12-21T13:46:06Z,2019-12-21T14:31:06Z,fc448035-8fc9-4893-b4eb-d1b90fc8742c,d733d4a9-080d-3593-b910-2366e652b7ea,8cddbb1b-7e2c-3bf0-957e-bc62c160cfc5,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,ambulatory,424441002,Prenatal initial visit,129.16,129.16,0.00,7.289200e+07,Normal pregnancy
34372,2a86f205-db88-4020-9746-74cf7248053e,1991-10-13T10:57:26Z,1991-10-14T10:57:26Z,ff6948b7-3d62-4bfc-bc7d-dbb6748ed0be,23834663-ed53-3da9-b330-d6e1ecb8428e,af42f305-3f0c-396d-8303-9cbdb900a516,4d71f845-a6a9-3c39-b242-14d25ef86a8d,inpatient,183452005,Encounter Inpatient,77.49,77.49,12.49,1.241710e+14,Chronic intractable migraine without aura
2017,669238d3-b3a0-438b-9fc8-d3b161e3dbee,1969-03-01T17:04:24Z,1969-03-01T19:59:24Z,47a91815-0dca-49df-b1dc-8bb88d8c1b38,ebc3f5c4-6700-34af-8323-85621c313726,eabb2bff-3216-34da-9f29-824dbca901c3,7c4411ce-02f1-39b5-b9ec-dfbea9ad3c1a,emergency,50849002,Emergency Room Admission,129.16,129.16,69.16,1.286130e+08,Seizure disorder


#### We need only a select number of columns. Also how about renaming them to be a bit descriptive

In [59]:
encounter=encounter.rename(columns={"PATIENT":"PATIENT_ID","ORGANIZATION":"ORG_ID","PROVIDER":"PROVIDER_ID","PAYER":"PAYER_ID","REASONDESCRIPTION":"REASON"})
encounter=encounter[["Id","START","STOP","PATIENT_ID","ORG_ID","PROVIDER_ID","PAYER_ID","DESCRIPTION","REASON","BASE_ENCOUNTER_COST","PAYER_COVERAGE","ENCOUNTERCLASS"]]
encounter.sample(3)

,Id,START,STOP,PATIENT_ID,ORG_ID,PROVIDER_ID,PAYER_ID,DESCRIPTION,REASON,BASE_ENCOUNTER_COST,PAYER_COVERAGE,ENCOUNTERCLASS
860,34404ede-f66d-40ed-8c31-88430e4cc1fc,2007-05-29T20:01:32Z,2007-05-29T20:31:32Z,2c634692-05d8-4122-9f46-dd9402e2cfc6,7b76c6b3-c413-3293-914a-fcf8195f32bc,b274d84c-d063-3cec-a8f1-f85c9bf03bb9,7c4411ce-02f1-39b5-b9ec-dfbea9ad3c1a,General examination of patient (procedure),NaN,129.16,129.16,wellness
27066,38f98ec0-5f6f-492b-a864-1cb41d225b17,1993-01-27T18:22:05Z,1993-01-27T18:56:05Z,19ad9612-4b4d-4a82-a323-c07168ec4d85,0b78995f-8b45-34d3-969d-afcc456bb1c7,89334094-3082-3984-8835-2f4bb972790b,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,General examination of patient (procedure),NaN,129.16,0.00,wellness
6864,8e3665de-bf71-41ae-83f3-95ede485bd0a,2001-03-12T05:08:34Z,2001-03-12T05:38:34Z,09616ead-22c8-4210-8cb9-2fdc28e043ca,1c9f06a8-38bb-3d07-99b2-176d626bc4c2,68aa1836-8435-393e-849e-7fec24d58199,5059a55e-5d6e-34d1-b6cb-d83d16e57bcf,General examination of patient (procedure),NaN,129.16,59.16,wellness


#### Now lets connect all the other elements and make one coherent data source

### I -  PATIENTS

In [60]:
patients=pd.read_csv(files["patients"])
patients.sample(3)

,Id,BIRTHDATE,DEATHDATE,SSN,DRIVERS,PASSPORT,PREFIX,FIRST,LAST,SUFFIX,...,BIRTHPLACE,ADDRESS,CITY,STATE,COUNTY,ZIP,LAT,LON,HEALTHCARE_EXPENSES,HEALTHCARE_COVERAGE
433,a49b8342-1954-48dc-9322-24bbfe3662c5,1913-05-30,1978-10-24,999-60-4614,S99966530,X32270562X,Mr.,William805,Kessler503,NaN,...,Boston Massachusetts US,185 Kuhic Lodge,Boston,Massachusetts,Suffolk County,2119.0,42.367481,-71.025350,1567509.30,5542.76
465,ab6a2662-f6d1-4da6-b3ce-3929d68650d7,1971-01-16,NaN,999-76-3317,S99978505,X28929072X,Mrs.,Miesha237,Wyman904,NaN,...,Harvard Massachusetts US,850 Thiel Road Unit 0,Westfield,Massachusetts,Hampden County,1086.0,42.090443,-72.792757,1127414.95,12398.31
1083,664dea73-3a66-4f67-bc88-300660a1dec9,1961-05-25,NaN,999-43-9671,S99964608,X29732912X,Mr.,Eugene421,Hintz995,NaN,...,Newton Massachusetts US,1076 Walker Alley,Grafton,Massachusetts,Worcester County,NaN,42.175492,-71.646736,180585.72,5696.16


#### Lets do some basic cleaning:

- Select only a few important columns
- Calculate the age of the patients
- Remove outliers (Consider assumption that age is < 100)
- Join the Patients first and last name

In [61]:
# SELECT ONLY REQUIRED COLUMNS
patients=patients[["Id","BIRTHDATE","FIRST","LAST","CITY"]]
patients.head()

,Id,BIRTHDATE,FIRST,LAST,CITY
0,1d604da9-9a81-4ba9-80c2-de3375d59b40,1989-05-25,José Eduardo181,Gómez206,Chicopee
1,034e9e3b-2def-4559-bb2a-7850888ae060,1983-11-14,Milo271,Feil794,Somerville
2,10339b10-3cd1-4ac3-ac13-ec26728cb592,1992-06-02,Jayson808,Fadel536,Chicopee
3,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,1978-05-27,Mariana775,Rutherford999,Lowell
4,f5dcd418-09fe-4a2f-baa0-3da800bd8c3a,1996-10-18,Gregorio366,Auer97,Boston


In [62]:
# CREATE A COLUMN AGE THAT CAN BE CALCULATED USING BIRTHDATE
from datetime import datetime
current_year=datetime.now().year

patients["BIRTHDATE"]=pd.to_datetime(patients["BIRTHDATE"],format="%Y-%m-%d")
patients["AGE"]=patients["BIRTHDATE"].apply(lambda x:current_year-x.year)
patients=patients[patients["AGE"]<100]
patients=patients.drop("BIRTHDATE",axis=1)
patients

,Id,FIRST,LAST,CITY,AGE
0,1d604da9-9a81-4ba9-80c2-de3375d59b40,José Eduardo181,Gómez206,Chicopee,31
1,034e9e3b-2def-4559-bb2a-7850888ae060,Milo271,Feil794,Somerville,37
2,10339b10-3cd1-4ac3-ac13-ec26728cb592,Jayson808,Fadel536,Chicopee,28
3,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,Mariana775,Rutherford999,Lowell,42
4,f5dcd418-09fe-4a2f-baa0-3da800bd8c3a,Gregorio366,Auer97,Boston,24
...,...,...,...,...,...
1163,30d83f4d-837c-488d-9748-5ee0fa49edd5,Napoleon578,Rice937,Yarmouth,55
1164,6c2b1cfd-887a-43f8-bee0-000ce2fc6ef0,Mercedes82,Shields502,Dennis,33
1165,09867b02-24b8-4896-b38b-86782dd64ef1,Laurence43,Weber641,Wilmington,63
1166,023a7d29-32b3-4db5-89c8-b88bd7582ec0,Conchita9,Peres371,Cambridge,98


In [63]:
# JOIN THE PATIENTS NAMES
patients=patients.rename(columns={"Id":"PATIENT_ID"})
patients["PATIENT_NAME"]=patients["FIRST"].str.cat(patients["LAST"],sep=" ")
patients=patients.drop(["FIRST","LAST"],axis=1)
patients=patients.set_index("PATIENT_ID")
patients

,CITY,AGE,PATIENT_NAME
PATIENT_ID,,,
1d604da9-9a81-4ba9-80c2-de3375d59b40,Chicopee,31,José Eduardo181 Gómez206
034e9e3b-2def-4559-bb2a-7850888ae060,Somerville,37,Milo271 Feil794
10339b10-3cd1-4ac3-ac13-ec26728cb592,Chicopee,28,Jayson808 Fadel536
8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,Lowell,42,Mariana775 Rutherford999
f5dcd418-09fe-4a2f-baa0-3da800bd8c3a,Boston,24,Gregorio366 Auer97
...,...,...,...
30d83f4d-837c-488d-9748-5ee0fa49edd5,Yarmouth,55,Napoleon578 Rice937
6c2b1cfd-887a-43f8-bee0-000ce2fc6ef0,Dennis,33,Mercedes82 Shields502
09867b02-24b8-4896-b38b-86782dd64ef1,Wilmington,63,Laurence43 Weber641


### Now let's integrate this with the encounter data

- We're Using 'join' in the this. By the way do you know whats the difference between join and merge ??

In [64]:
encounter=encounter.join(patients,on="PATIENT_ID")
encounter

,Id,START,STOP,PATIENT_ID,ORG_ID,PROVIDER_ID,PAYER_ID,DESCRIPTION,REASON,BASE_ENCOUNTER_COST,PAYER_COVERAGE,ENCOUNTERCLASS,CITY,AGE,PATIENT_NAME
0,d0c40d10-8d87-447e-836e-99d26ad52ea5,2010-01-23T17:45:28Z,2010-01-23T18:10:28Z,034e9e3b-2def-4559-bb2a-7850888ae060,e002090d-4e92-300e-b41e-7d1f21dee4c6,e6283e46-fd81-3611-9459-0edb1c3da357,6e2f1a2d-27bd-3701-8d08-dae202c58632,Encounter for symptom,Acute bronchitis (disorder),129.16,54.16,ambulatory,Somerville,37.0,Milo271 Feil794
1,e88bc3a9-007c-405e-aabc-792a38f4aa2b,2012-01-23T17:45:28Z,2012-01-23T18:00:28Z,034e9e3b-2def-4559-bb2a-7850888ae060,772ee193-bb9f-30eb-9939-21e86c8e4da5,6f1d59a7-a5bd-3cf9-9671-5bad2f351c28,6e2f1a2d-27bd-3701-8d08-dae202c58632,General examination of patient (procedure),NaN,129.16,129.16,wellness,Somerville,37.0,Milo271 Feil794
2,8f104aa7-4ca9-4473-885a-bba2437df588,2001-05-01T15:02:18Z,2001-05-01T15:17:18Z,1d604da9-9a81-4ba9-80c2-de3375d59b40,5d4b9df1-93ae-3bc9-b680-03249990e558,af01a385-31d3-3c77-8fdb-2867fe88df2f,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,Encounter for symptom,Sinusitis (disorder),129.16,0.00,ambulatory,Chicopee,31.0,José Eduardo181 Gómez206
3,b85c339a-6076-43ed-b9d0-9cf013dec49d,2011-07-28T15:02:18Z,2011-07-28T15:17:18Z,1d604da9-9a81-4ba9-80c2-de3375d59b40,3dc9bb2d-5d66-3e61-bf9a-e234c6433577,bb17e691-262b-3546-93d5-d88e7de93246,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,General examination of patient (procedure),NaN,129.16,0.00,wellness,Chicopee,31.0,José Eduardo181 Gómez206
4,dae2b7cb-1316-4b78-954f-fa610a6c6d0e,2010-07-27T12:58:08Z,2010-07-27T13:28:08Z,10339b10-3cd1-4ac3-ac13-ec26728cb592,b03dba4f-892f-365c-bfd1-bfcfa7a98d5d,7ed6b84a-b847-3744-9d42-15c42297a0c2,d47b3510-2895-3b70-9897-342d681c769d,General examination of patient (procedure),NaN,129.16,129.16,wellness,Chicopee,28.0,Jayson808 Fadel536
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53341,bd3bfc3c-4f52-4d9c-aa01-60a084b2ddcd,2015-06-20T17:27:52Z,2015-06-20T17:57:52Z,fc817953-cc8b-45db-9c85-7c0ced8fa90d,a44b37da-aa9b-37f0-993e-26e58974e086,ff678a28-0ee8-379c-80ab-2dc790cf67dd,7caa7254-5050-3b5e-9eae-bd5ea30e809c,General examination of patient (procedure),NaN,129.16,129.16,wellness,NaN,NaN,NaN
53342,59727285-d30f-44e2-bcda-eac21ce07269,2016-06-25T17:27:52Z,2016-06-25T17:42:52Z,fc817953-cc8b-45db-9c85-7c0ced8fa90d,a44b37da-aa9b-37f0-993e-26e58974e086,ff678a28-0ee8-379c-80ab-2dc790cf67dd,7caa7254-5050-3b5e-9eae-bd5ea30e809c,General examination of patient (procedure),NaN,129.16,129.16,wellness,NaN,NaN,NaN
53343,957b13d5-49fa-4518-8c25-6eaa45e6a7d1,2017-07-01T17:27:52Z,2017-07-01T17:57:52Z,fc817953-cc8b-45db-9c85-7c0ced8fa90d,a44b37da-aa9b-37f0-993e-26e58974e086,ff678a28-0ee8-379c-80ab-2dc790cf67dd,7caa7254-5050-3b5e-9eae-bd5ea30e809c,General examination of patient (procedure),NaN,129.16,129.16,wellness,NaN,NaN,NaN
53344,5529a0d5-a24c-42b4-a32a-729e4d1f19ef,2018-07-07T17:27:52Z,2018-07-07T17:42:52Z,fc817953-cc8b-45db-9c85-7c0ced8fa90d,a44b37da-aa9b-37f0-993e-26e58974e086,ff678a28-0ee8-379c-80ab-2dc790cf67dd,7caa7254-5050-3b5e-9eae-bd5ea30e809c,General examination of patient (procedure),NaN,129.16,129.16,wellness,NaN,NaN,NaN


####  II HOSPITALS

In [65]:
# READ THE ORGANISATION FILES
organisations=pd.read_csv(files["organizations"])
organisations.sample(3)

,Id,NAME,ADDRESS,CITY,STATE,ZIP,LAT,LON,PHONE,REVENUE,UTILIZATION
963,3e4299d5-499d-3fb7-a4d7-1b4df8e29c5a,STEWARD MEDICAL GROUP INC,11 NEVINS ST,BRIGHTON,MA,02135-3514,42.331960,-71.020173,617-789-2045,6458.00,50
480,e8d33a84-cad5-306a-9982-b011e4da175e,PCP77608,185 LINCOLN ST,HINGHAM,MA,02043-1743,42.224319,-70.880673,781-740-2349,11495.24,90
753,1ac92984-69ab-3c07-9b52-73736930a1b5,MERRIMACK VALLEY PET PC,70 E ST,METHUEN,MA,01844-4597,42.742751,-71.178588,978-689-4738,5424.72,42


In [66]:
# SELECT A FEW COLUMNS AND RENAME THEM
organisations=organisations[["Id","NAME","CITY","REVENUE"]]
organisations=organisations.rename(columns={"NAME":"ORG_NAME","Id":"ORG_ID","CITY":"ORG_CITY"})
organisations=organisations.set_index("ORG_ID")
organisations.sample(3)

,ORG_NAME,ORG_CITY,REVENUE
ORG_ID,,,
903d3e6b-ace5-33a8-ab58-63f9fc4fc497,MASSACHUSETTS GENERAL PHYSICIANS ORGANIZATION ...,SOMERVILLE,8395.40
ae67223f-808e-33d3-a83e-066deebf35ed,VA Central Western Massachusetts Healthcare Sy...,Leeds,17436.60
ef6ab57c-ed94-3dbe-9861-812d515918b3,CAPE COD HOSPITAL,HYANNIS,226779.11


In [67]:
# JOIN TO  ENCOUNTERS
encounter=encounter.join(organisations,on="ORG_ID")
encounter

,Id,START,STOP,PATIENT_ID,ORG_ID,PROVIDER_ID,PAYER_ID,DESCRIPTION,REASON,BASE_ENCOUNTER_COST,PAYER_COVERAGE,ENCOUNTERCLASS,CITY,AGE,PATIENT_NAME,ORG_NAME,ORG_CITY,REVENUE
0,d0c40d10-8d87-447e-836e-99d26ad52ea5,2010-01-23T17:45:28Z,2010-01-23T18:10:28Z,034e9e3b-2def-4559-bb2a-7850888ae060,e002090d-4e92-300e-b41e-7d1f21dee4c6,e6283e46-fd81-3611-9459-0edb1c3da357,6e2f1a2d-27bd-3701-8d08-dae202c58632,Encounter for symptom,Acute bronchitis (disorder),129.16,54.16,ambulatory,Somerville,37.0,Milo271 Feil794,CAMBRIDGE HEALTH ALLIANCE,CAMBRIDGE,406053.19
1,e88bc3a9-007c-405e-aabc-792a38f4aa2b,2012-01-23T17:45:28Z,2012-01-23T18:00:28Z,034e9e3b-2def-4559-bb2a-7850888ae060,772ee193-bb9f-30eb-9939-21e86c8e4da5,6f1d59a7-a5bd-3cf9-9671-5bad2f351c28,6e2f1a2d-27bd-3701-8d08-dae202c58632,General examination of patient (procedure),NaN,129.16,129.16,wellness,Somerville,37.0,Milo271 Feil794,PCP20565,CAMBRIDGE,4262.28
2,8f104aa7-4ca9-4473-885a-bba2437df588,2001-05-01T15:02:18Z,2001-05-01T15:17:18Z,1d604da9-9a81-4ba9-80c2-de3375d59b40,5d4b9df1-93ae-3bc9-b680-03249990e558,af01a385-31d3-3c77-8fdb-2867fe88df2f,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,Encounter for symptom,Sinusitis (disorder),129.16,0.00,ambulatory,Chicopee,31.0,José Eduardo181 Gómez206,HOLYOKE MEDICAL CENTER,HOLYOKE,327601.37
3,b85c339a-6076-43ed-b9d0-9cf013dec49d,2011-07-28T15:02:18Z,2011-07-28T15:17:18Z,1d604da9-9a81-4ba9-80c2-de3375d59b40,3dc9bb2d-5d66-3e61-bf9a-e234c6433577,bb17e691-262b-3546-93d5-d88e7de93246,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,General examination of patient (procedure),NaN,129.16,0.00,wellness,Chicopee,31.0,José Eduardo181 Gómez206,PCP12399,SOUTH HADLEY,7491.28
4,dae2b7cb-1316-4b78-954f-fa610a6c6d0e,2010-07-27T12:58:08Z,2010-07-27T13:28:08Z,10339b10-3cd1-4ac3-ac13-ec26728cb592,b03dba4f-892f-365c-bfd1-bfcfa7a98d5d,7ed6b84a-b847-3744-9d42-15c42297a0c2,d47b3510-2895-3b70-9897-342d681c769d,General examination of patient (procedure),NaN,129.16,129.16,wellness,Chicopee,28.0,Jayson808 Fadel536,WESTERN MASS PHYSICIAN ASSOCIATES INC,CHICOPEE,7620.44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53341,bd3bfc3c-4f52-4d9c-aa01-60a084b2ddcd,2015-06-20T17:27:52Z,2015-06-20T17:57:52Z,fc817953-cc8b-45db-9c85-7c0ced8fa90d,a44b37da-aa9b-37f0-993e-26e58974e086,ff678a28-0ee8-379c-80ab-2dc790cf67dd,7caa7254-5050-3b5e-9eae-bd5ea30e809c,General examination of patient (procedure),NaN,129.16,129.16,wellness,NaN,NaN,NaN,PCP27951,WESTWOOD,12141.04
53342,59727285-d30f-44e2-bcda-eac21ce07269,2016-06-25T17:27:52Z,2016-06-25T17:42:52Z,fc817953-cc8b-45db-9c85-7c0ced8fa90d,a44b37da-aa9b-37f0-993e-26e58974e086,ff678a28-0ee8-379c-80ab-2dc790cf67dd,7caa7254-5050-3b5e-9eae-bd5ea30e809c,General examination of patient (procedure),NaN,129.16,129.16,wellness,NaN,NaN,NaN,PCP27951,WESTWOOD,12141.04
53343,957b13d5-49fa-4518-8c25-6eaa45e6a7d1,2017-07-01T17:27:52Z,2017-07-01T17:57:52Z,fc817953-cc8b-45db-9c85-7c0ced8fa90d,a44b37da-aa9b-37f0-993e-26e58974e086,ff678a28-0ee8-379c-80ab-2dc790cf67dd,7caa7254-5050-3b5e-9eae-bd5ea30e809c,General examination of patient (procedure),NaN,129.16,129.16,wellness,NaN,NaN,NaN,PCP27951,WESTWOOD,12141.04
53344,5529a0d5-a24c-42b4-a32a-729e4d1f19ef,2018-07-07T17:27:52Z,2018-07-07T17:42:52Z,fc817953-cc8b-45db-9c85-7c0ced8fa90d,a44b37da-aa9b-37f0-993e-26e58974e086,ff678a28-0ee8-379c-80ab-2dc790cf67dd,7caa7254-5050-3b5e-9eae-bd5ea30e809c,General examination of patient (procedure),NaN,129.16,129.16,wellness,NaN,NaN,NaN,PCP27951,WESTWOOD,12141.04


#### III HEALTHCARE PROFESSIONALS

In [68]:
providers=pd.read_csv(files["providers"])
providers.sample(3)

,Id,ORGANIZATION,NAME,GENDER,SPECIALITY,ADDRESS,CITY,STATE,ZIP,LAT,LON,UTILIZATION
1505,4f079296-dcf4-3501-9f2e-ab73dec6b8f0,a765c05e-ac8a-391d-bab9-3d70749cd6e8,Sylvester827 Block661,M,NURSE PRACTITIONER,585 LEBANON ST,MELROSE,MA,02176-3225,42.455723,-71.059019,0
1824,675d796b-1724-334c-8e30-6c4b5bf5418e,819cb4e2-fb1c-3917-80c9-fdb705d2cd19,Reggie481 Fadel536,M,NEPHROLOGY,1575 CAMBRIDGE ST,CAMBRIDGE,MA,02138-4308,42.376043,-71.118680,0
1256,e341d173-1d9e-3285-8808-19d504a70d80,c1414e2c-48df-3428-92a4-ac36114a2a45,Jude172 Franecki195,F,GENERAL PRACTICE,160 ROUTE 137,EAST HARWICH,MA,02645-1316,41.708597,-70.031856,230


In [69]:
providers=providers[["Id","NAME","GENDER","SPECIALITY"]]
providers=providers.rename(columns={"NAME":"PROVIDER_NAME","Id":"PROVIDER_ID"})
providers=providers.set_index("PROVIDER_ID")
providers.sample(3)

,PROVIDER_NAME,GENDER,SPECIALITY
PROVIDER_ID,,,
2840e8f1-5624-3913-9996-f7639e07e76b,Nelly786 Jaskolski867,F,OTOLARYNGOLOGY
f785281d-56f1-3635-9d06-946eddb1ec63,Demarcus108 Jast432,M,CHIROPRACTIC
25e15180-dcb5-3c74-a8f0-5be7881fa41d,Danna372 Okuneva707,F,OTOLARYNGOLOGY


In [70]:
# join provider and hospitals
encounter=encounter.join(providers,on="PROVIDER_ID")
encounter.sample(3)

,Id,START,STOP,PATIENT_ID,ORG_ID,PROVIDER_ID,PAYER_ID,DESCRIPTION,REASON,BASE_ENCOUNTER_COST,...,ENCOUNTERCLASS,CITY,AGE,PATIENT_NAME,ORG_NAME,ORG_CITY,REVENUE,PROVIDER_NAME,GENDER,SPECIALITY
2539,40a0cbb2-de8f-48fe-9f77-1315c17cb4c3,2016-03-04T04:10:32Z,2016-03-04T04:25:32Z,dadd331c-6143-41d1-bffb-d34b321ed157,a3d316af-a158-37b2-bcc6-b014ddb08fa9,c421ce4f-75c3-317c-9754-7544158afca2,5059a55e-5d6e-34d1-b6cb-d83d16e57bcf,General examination of patient (procedure),NaN,129.16,...,wellness,Dedham,44.0,Shirley182 Borer986,PCP65288,DEDHAM,6716.32,Gil594 Schultz619,M,GENERAL PRACTICE
19725,2369f5c0-98b0-4565-8053-855e7f2c140c,2011-04-09T13:45:16Z,2011-04-09T14:15:16Z,0e866809-bc7a-4014-850d-06228ed80226,5ebb2110-3a7b-32c6-adf5-60f42b27af76,7153c562-d21b-3e10-a122-88ba818da40a,7c4411ce-02f1-39b5-b9ec-dfbea9ad3c1a,General examination of patient (procedure),NaN,129.16,...,wellness,Holyoke,65.0,Sheldon401 Powlowski563,PCP30214,EASTHAMPTON,6974.64,Malia984 O'Kon634,F,GENERAL PRACTICE
34447,b5fc3e8d-6070-4021-9107-ce76281f35ea,2017-06-17T10:57:26Z,2017-06-17T11:12:26Z,ff6948b7-3d62-4bfc-bc7d-dbb6748ed0be,23834663-ed53-3da9-b330-d6e1ecb8428e,af42f305-3f0c-396d-8303-9cbdb900a516,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,Encounter for problem (procedure),NaN,129.16,...,outpatient,Fall River,43.0,Kris249 Champlin946,SOUTHCOAST HOSPITAL GROUP INC,FALL RIVER,222412.59,Myong12 Heidenreich818,F,GENERAL PRACTICE


#### IV  - INSURANCE COMPANIES

In [71]:
payers=pd.read_csv(files["payers"])
payers.sample(3)

,Id,NAME,ADDRESS,CITY,STATE_HEADQUARTERED,ZIP,PHONE,AMOUNT_COVERED,AMOUNT_UNCOVERED,REVENUE,...,UNCOVERED_ENCOUNTERS,COVERED_MEDICATIONS,UNCOVERED_MEDICATIONS,COVERED_PROCEDURES,UNCOVERED_PROCEDURES,COVERED_IMMUNIZATIONS,UNCOVERED_IMMUNIZATIONS,UNIQUE_CUSTOMERS,QOLS_AVG,MEMBER_MONTHS
2,7c4411ce-02f1-39b5-b9ec-dfbea9ad3c1a,Medicaid,7500 Security Blvd,Baltimore,MD,21244.0,1-877-267-2323,3636994.03,1814967.93,20365500.0,...,0,6382,0,47611,0,7022,0,507,0.812137,84276
1,7caa7254-5050-3b5e-9eae-bd5ea30e809c,Medicare,7500 Security Blvd,Baltimore,MD,21244.0,1-800-633-4227,4756720.13,890394.37,7122250.0,...,0,13471,0,7383,0,2557,0,198,0.786223,29760
3,d47b3510-2895-3b70-9897-342d681c769d,Humana,500 West Main St,Louisville,KY,40018.0,1-844-330-7799,716971.60,842531.43,118500200.0,...,1796,0,4985,12718,550,6832,195,281,0.911843,70308


In [72]:
payers=payers[["NAME","Id"]]
payers=payers.rename(columns={"NAME":"PAYER_NAME","Id":"PAYER_ID"})
payers=payers.set_index("PAYER_ID")
payers

,PAYER_NAME
PAYER_ID,
b3221cfc-24fb-339e-823d-bc4136cbc4ed,Dual Eligible
7caa7254-5050-3b5e-9eae-bd5ea30e809c,Medicare
7c4411ce-02f1-39b5-b9ec-dfbea9ad3c1a,Medicaid
d47b3510-2895-3b70-9897-342d681c769d,Humana
6e2f1a2d-27bd-3701-8d08-dae202c58632,Blue Cross Blue Shield
5059a55e-5d6e-34d1-b6cb-d83d16e57bcf,UnitedHealthcare
4d71f845-a6a9-3c39-b242-14d25ef86a8d,Aetna
047f6ec3-6215-35eb-9608-f9dda363a44c,Cigna Health
42c4fca7-f8a9-3cd1-982a-dd9751bf3e2a,Anthem


In [73]:
# JOIN ON ENCOUNTERS
encounter=encounter.join(payers,on="PAYER_ID")
encounter.sample(3)

,Id,START,STOP,PATIENT_ID,ORG_ID,PROVIDER_ID,PAYER_ID,DESCRIPTION,REASON,BASE_ENCOUNTER_COST,...,CITY,AGE,PATIENT_NAME,ORG_NAME,ORG_CITY,REVENUE,PROVIDER_NAME,GENDER,SPECIALITY,PAYER_NAME
38257,983d84b0-e93e-42f3-a1c8-06fef1f7d2a0,2013-03-04T08:54:26Z,2013-03-04T09:24:26Z,04db6603-0017-4cc6-a46d-6df577b0a10d,6ffe62de-5d90-3cc7-be16-643a7496c817,bd8ee454-312d-3bb6-9d18-95f07366c516,7c4411ce-02f1-39b5-b9ec-dfbea9ad3c1a,General examination of patient (procedure),NaN,129.16,...,Leverett,48.0,Angella518 Reynolds644,PCP8458,SUNDERLAND,7232.96,Augustine565 Fisher429,M,GENERAL PRACTICE,Medicaid
32267,a0643b55-9cd6-4e02-8739-05cfe2bc6c87,2015-11-12T21:42:25Z,2015-11-12T22:12:25Z,4f0cc2f3-8492-41c9-ad38-f01219739df0,a0a14713-50c4-3818-b0cb-8c2047f1458c,341e9a0c-e102-3e0c-b5df-3370a2ad2c61,42c4fca7-f8a9-3cd1-982a-dd9751bf3e2a,General examination of patient (procedure),NaN,129.16,...,Plymouth,56.0,Analisa263 Kiehn525,BAY STATE PHYSICAL THERAPY OF RANDOLPH PC,CEDARVILLE,13303.48,Tobias236 Bechtelar572,M,GENERAL PRACTICE,Anthem
21457,91c8ef6a-3f2f-4842-afd1-7f27e3ec63ac,1997-08-13T21:29:36Z,1997-08-13T21:59:36Z,85a07bb9-eb47-4fb3-8f96-c0a77f4821c6,7dbf8d9d-fc47-39df-9e54-649d6eb4f8d3,253874bc-8d3a-329d-8d8b-3f258db62074,d47b3510-2895-3b70-9897-342d681c769d,General examination of patient (procedure),NaN,129.16,...,Boston,48.0,Yuko264 O'Conner199,PCP156597,BRIGHTON,7232.96,Porsche32 Lowe577,F,GENERAL PRACTICE,Humana


### Lets clean up the Encounter Table a little bit

In [96]:
encounter.columns

Index(['Id', 'START', 'STOP', 'PATIENT_NAME', 'AGE', 'CITY', 'DESCRIPTION',
       'REASON', 'ENCOUNTERCLASS', 'BASE_ENCOUNTER_COST', 'PAYER_COVERAGE',
       'ORG_NAME', 'ORG_CITY', 'REVENUE', 'PROVIDER_NAME', 'SPECIALITY',
       'PAYER_NAME', 'YEAR'],
      dtype='object')

In [97]:
# only select a few columns in it
encounter=encounter[["Id","START","STOP","PATIENT_NAME","AGE","CITY",
                     "DESCRIPTION","REASON","ENCOUNTERCLASS",
                     "BASE_ENCOUNTER_COST","PAYER_COVERAGE",
                     "ORG_NAME","ORG_CITY","REVENUE",
                     "PROVIDER_NAME","SPECIALITY","PAYER_NAME"]]
encounter.sample(3)

,Id,START,STOP,PATIENT_NAME,AGE,CITY,DESCRIPTION,REASON,ENCOUNTERCLASS,BASE_ENCOUNTER_COST,PAYER_COVERAGE,ORG_NAME,ORG_CITY,REVENUE,PROVIDER_NAME,SPECIALITY,PAYER_NAME
8545,abf29f17-7062-45a2-8c9a-2c3e1b9efd6f,2012-11-15 07:10:29+00:00,2012-11-15T07:40:29Z,Brett333 Mante251,66.0,Barnstable,General examination of patient (procedure),NaN,wellness,129.16,129.16,Hyannis Outpatient Clinic,Hyannis,83101.40,Leonor133 Nájera755,GENERAL PRACTICE,Aetna
53146,11da5c9b-aa46-4e92-a13d-4ff2a2298174,2018-12-17 05:04:13+00:00,2018-12-17T05:19:13Z,Laverne101 Hane680,43.0,Beverly,Encounter for check up (procedure),NaN,outpatient,129.16,0.00,BEVERLY HOSPITAL CORPORATION,BEVERLY,153829.53,Mari763 Reynolds644,GENERAL PRACTICE,NO_INSURANCE
45481,063a7f6b-3698-48ef-a357-528786af036d,2011-12-10 12:52:02+00:00,2011-12-10T13:40:02Z,Phung894 Kihn564,22.0,Southampton,Allergic disorder initial assessment,NaN,ambulatory,129.16,54.16,COOLEY DICKINSON HOSPITAL INC THE,NORTHAMPTON,207766.77,Guadalupe206 Valenzuela371,GENERAL PRACTICE,Blue Cross Blue Shield


In [101]:
encounter["START"]=pd.to_datetime(encounter["START"],format="%Y-%m-%d")
encounter["YEAR"]=encounter["START"].apply(lambda x:x.year)
encounter.sample(3)

/home/riz-ryu/miniconda3/envs/stm/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Id,START,STOP,PATIENT_NAME,AGE,CITY,DESCRIPTION,REASON,ENCOUNTERCLASS,BASE_ENCOUNTER_COST,PAYER_COVERAGE,ORG_NAME,ORG_CITY,REVENUE,PROVIDER_NAME,SPECIALITY,PAYER_NAME,YEAR
1314,56c58193-8251-4ec1-8e04-e086f0f539ad,1987-08-02 05:28:51+00:00,1987-08-02T05:43:51Z,Mabel261 Larkin917,34.0,Framingham,Well child visit (procedure),NaN,wellness,129.16,54.16,MINUTECLINIC DIAGNOSTIC OF MASSACHUSETTS LLC,SUDBURY,1679.08,Kenny207 Ward668,GENERAL PRACTICE,Blue Cross Blue Shield,1987
20350,9a6b8a5f-43a0-4a83-b022-476b109a7ad4,1974-07-11 04:21:46+00:00,1974-07-11T04:51:46Z,Dannette613 Bartoletti50,55.0,Holyoke,Well child visit (procedure),NaN,wellness,129.16,49.16,PCP29548,HOLYOKE,8137.08,Nenita289 Dare640,GENERAL PRACTICE,Humana,1974
52750,562f0b12-411f-4393-8f1a-c98a9b4b0f71,2011-08-28 04:37:39+00:00,2011-08-28T05:07:39Z,Lashawn862 Koepp521,79.0,Easthampton,General examination of patient (procedure),NaN,wellness,129.16,129.16,NORTHAMPTON RADIOLOGIC ASSOCIATES INC.,FLORENCE,8782.88,Adria871 Boehm581,GENERAL PRACTICE,Medicare,2011


In [103]:
encounter = encounter[encounter["YEAR"]>=2010]

#### Lets answer certain general questions for now

### I - How has the year wise trend been with the Insurance companies ? 

We're going to use a simple animation using plotly

In [109]:
counts = encounter.groupby(['PAYER_NAME','YEAR']).count().reset_index()
px.bar(counts,x='PAYER_NAME',y='Id',animation_frame='YEAR')

### II - How has the year wise trend been in the number of patient visits ?

This is bit more complex animation with a play button

In [112]:
no_cases=encounter.groupby(["YEAR"]).count()["Id"].reset_index()
no_cases

,YEAR,Id
0,2010,2083
1,2011,2979
2,2012,2890
3,2013,3058
4,2014,3178
5,2015,3063
6,2016,3148
7,2017,3153
8,2018,3090
9,2019,2919


#### We need to create a year wise trend line animation

- x- axis: year

- y-axis : count

For this we need : 
- range of values on the x axis and y axis
- starting values for x and y
- x and y values for each year

And for the plotly chart we need to define:
- Layout (Buttons)
- Frames (x and y value of each year)
- data: starting and y values

In [114]:
# find the range of x values and y values
x_range = [no_cases["YEAR"].min(),no_cases["YEAR"].max()]
y_range = [no_cases["YEAR"].min(),no_cases["Id"].max()]

In [117]:
# Starting x values and start y values
start_x = no_cases["YEAR"].iloc[0]
start_y = no_cases["Id"].iloc[0]

data = go.Scatter(x=[start_x,start_x],y=[start_y,start_y])

In [125]:
# individual x and y values
frames = []
for i in range(2,len(no_cases)+1):
    frame = go.Scatter(x=no_cases["YEAR"].iloc[:i],y=no_cases["Id"].iloc[:i])
    frame = go.Frame(data=[frame])
    frames.append(frame)
    


In [126]:
button = {"label":"Play",
          "method":"animate",
         "args":[None]
         }
update_menus = {"type":"buttons",
         "buttons":[button],
          }
layout=go.Layout(xaxis=dict(range=x_range),yaxis=dict(range=y_range),updatemenus=[update_menus])

In [127]:
fig=go.Figure(data=[data],layout=layout,frames=frames)
fig.show()

## Now consider the stakeholders involved in this scenario, and try to determine what analysis of data  will be useful for each of them.

#### Lets take the example of the hospitals here. What all data can be useful for one hospital ?
The hospital has data records for the past 10 years. Thequestions asked can include
- How many patients came in each year ? 
- What is the most popular reason for a patient to visit ?
- Which insurance company is the most popular ?
- What are the common medications/tests prescribed ? 
- What doctors/ speciality is most in demand ?
- How many emergency cases are coming in ?
- Whats the age group of the patients ?

Depending on the answers to these questions 
the hospital can take decisions like:
- What speciality of doctors need to be hired ? 
- What medicines/ medical equipment should be ordered ? 
- Which Insurance company can be a potential partner ? 
- How stringent should the emergency preparedness be ?

### Lets look at how many hospitals do we have

In [104]:
encounter["ORG_NAME"].value_counts()

HALLMARK HEALTH SYSTEM                               1324
VA Boston Healthcare System  Jamaica Plain Campus    1196
METROWEST MEDICAL CENTER                              712
NEWTON-WELLESLEY HOSPITAL                             691
CAMBRIDGE HEALTH ALLIANCE                             689
                                                     ... 
HAWTHORN MEDICAL URGENT CARE CENTER                     2
HAMPDEN COUNTY URGENT MEDICAL CARE CENTER               2
MASHPEE FAMILY MEDICINE                                 1
TRU MEDICAL WALK IN CENTER                              1
FALL RIVER MEDICAL EMERGENCY WALK IN CLINIC             1
Name: ORG_NAME, Length: 883, dtype: int64

### Lets choose a random hospital. We're going with HALLMARK HEALTHCARE

In [128]:
df=encounter[encounter["ORG_NAME"]=="HALLMARK HEALTH SYSTEM"]
df

,Id,START,STOP,PATIENT_NAME,AGE,CITY,DESCRIPTION,REASON,ENCOUNTERCLASS,BASE_ENCOUNTER_COST,PAYER_COVERAGE,ORG_NAME,ORG_CITY,REVENUE,PROVIDER_NAME,SPECIALITY,PAYER_NAME,YEAR
632,0f2eccff-7741-4ff2-a13a-656cbd78394a,2010-12-17 14:19:13+00:00,2010-12-17T19:34:13Z,Lisbeth69 Hand679,37.0,Malden,Prenatal initial visit,Normal pregnancy,ambulatory,129.16,64.16,HALLMARK HEALTH SYSTEM,MELROSE,561587.35,Vern731 Powlowski563,GENERAL PRACTICE,Aetna,2010
633,a8f5aec6-5b20-4c6e-b79b-4adeac18f978,2011-01-14 14:19:13+00:00,2011-01-14T15:19:13Z,Lisbeth69 Hand679,37.0,Malden,Prenatal visit,Normal pregnancy,ambulatory,129.16,64.16,HALLMARK HEALTH SYSTEM,MELROSE,561587.35,Vern731 Powlowski563,GENERAL PRACTICE,Aetna,2011
634,1bd80074-32d8-454f-916a-4aff32595a81,2011-02-11 14:19:13+00:00,2011-02-11T15:34:13Z,Lisbeth69 Hand679,37.0,Malden,Prenatal visit,Normal pregnancy,ambulatory,129.16,64.16,HALLMARK HEALTH SYSTEM,MELROSE,561587.35,Vern731 Powlowski563,GENERAL PRACTICE,Aetna,2011
635,2e9c5abc-f9d1-45fa-9a98-69f99fb245d9,2011-03-11 14:19:13+00:00,2011-03-11T15:04:13Z,Lisbeth69 Hand679,37.0,Malden,Prenatal visit,Normal pregnancy,ambulatory,129.16,64.16,HALLMARK HEALTH SYSTEM,MELROSE,561587.35,Vern731 Powlowski563,GENERAL PRACTICE,Aetna,2011
636,61b102d3-fa48-49a8-b6bc-9214a052b6df,2011-04-08 14:19:13+00:00,2011-04-08T15:04:13Z,Lisbeth69 Hand679,37.0,Malden,Prenatal visit,Normal pregnancy,ambulatory,129.16,64.16,HALLMARK HEALTH SYSTEM,MELROSE,561587.35,Vern731 Powlowski563,GENERAL PRACTICE,Aetna,2011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52475,eb1b8c76-56f4-46ea-9e5f-5f0363db9858,2013-01-07 12:14:09+00:00,2013-01-07T12:29:09Z,Hubert238 Kihn564,12.0,Boston,Encounter for check up (procedure),NaN,outpatient,129.16,0.00,HALLMARK HEALTH SYSTEM,MELROSE,561587.35,Vern731 Powlowski563,GENERAL PRACTICE,UnitedHealthcare,2013
52476,fbb8bdc8-f506-4c1f-b899-0da2b4859a1a,2013-01-06 12:14:09+00:00,2013-01-06T12:29:09Z,Hubert238 Kihn564,12.0,Boston,Encounter for symptom,Acute bacterial sinusitis (disorder),ambulatory,129.16,59.16,HALLMARK HEALTH SYSTEM,MELROSE,561587.35,Vern731 Powlowski563,GENERAL PRACTICE,UnitedHealthcare,2013
52478,b35110b5-20b7-4ebf-84c5-62bf97ed94a3,2013-08-14 12:14:09+00:00,2013-08-14T12:29:09Z,Hubert238 Kihn564,12.0,Boston,Encounter for symptom,Viral sinusitis (disorder),ambulatory,129.16,59.16,HALLMARK HEALTH SYSTEM,MELROSE,561587.35,Vern731 Powlowski563,GENERAL PRACTICE,UnitedHealthcare,2013
52479,9a384fbd-3e24-41e6-b338-330a6efdddeb,2013-12-27 12:14:09+00:00,2013-12-27T12:29:09Z,Hubert238 Kihn564,12.0,Boston,Encounter for symptom,Viral sinusitis (disorder),ambulatory,129.16,59.16,HALLMARK HEALTH SYSTEM,MELROSE,561587.35,Vern731 Powlowski563,GENERAL PRACTICE,UnitedHealthcare,2013


#### Lets do a simple visualisation. 
We'd like to know what was the distribution of the number of cases covered by each insurance agency
Lets create a stacked chart

In [136]:
#group by year and payer name and take a count.
payer_distribution=df.groupby(["YEAR","PAYER_NAME"]).count().reset_index()
px.bar(payer_distribution,x="YEAR",y="Id",hover_data=["PAYER_NAME"],labels={"Id":"num_visits"})

#### Now let's try to find the popular reasons for which patients visited the hospital
We're going to use subplots for this
We need: 
- number of rows 
- number of columns
- and what each figure is going be (specs)

11

In [151]:
cols=1
rows = len(range(df["YEAR"].min(),df["YEAR"].max()+1))
specs = [[{"type":"pie"}]for i in range(rows)]
column_widths=[0.9]

In [152]:
fig = make_subplots(rows=rows,cols=cols,specs=specs,column_widths=column_widths)

In [153]:
row=1
# first group by year and then for each group we're going to take the value counts for the reason
for year,group in df.groupby(["YEAR"]):
    counts=group["REASON"].value_counts()
    fig.append_trace(go.Pie(title=year,labels=counts.index,values=counts.values,textinfo="none"),row=row,col=1)
    row+=1
fig.update_layout(height=4000, width=1000)
fig.show()

#### Now can we apply the same to find out the age distribution of the patients in every year?

In [154]:
row=1
fig=make_subplots(rows=11,cols=1,specs=[[{"type": "histogram"}],
                           [{"type": "histogram"}],
                           [{"type": "histogram"}],
                        [{"type": "histogram"}],
                           [{"type": "histogram"}],
                           [{"type": "histogram"}],
                           [{"type": "histogram"}],
                           [{"type": "histogram"}],
                           [{"type": "histogram"}],
                           [{"type": "histogram"}],
                            [{"type": "histogram"}],])

for year,group in df.groupby("YEAR"):
    fig.append_trace(go.Histogram(x=group["AGE"],name=year),row=row,col=1,)
    row+=1
fig.update_layout(height=2000,width=1000)
fig.show()

#### Practise:  Now try finding the number of visits from each city in each year (Lets just do it for 5 cities)

In [155]:
row=1
fig=make_subplots(rows=5,cols=1,specs=[[{"type": "bar"}],
                           [{"type": "bar"}],
                           [{"type": "bar"}],
                        [{"type": "bar"}],
                           [{"type": "bar"}]])
for city,group in df.groupby(["CITY"]):
    years=df["YEAR"].value_counts()
    fig.append_trace(go.Bar(x=years.index,y=years.values,name=city),row=row,col=1)
    row+=1
    if row>=5:
        break

fig.update_layout(height=1000,width=800)
fig.show()





### Takeaways and assignments:
Use the same logic to analyse and create visualisations from the point of view of some other stakeholder. Think about other possibilities. Get creative